# Zset

This page focuses on the `zset` redis data structure. It discusses all the interesting ways to work with `zset`-s.

Is a data structure where the number (score) corresponds to the value. This allows you to select values by the range of their scores.

## Examples

This section focuses on creating examples that we will use to show the possibilities of `zset`. It uses Python scripts, but the same result can be achieved using bash.

In [1]:
import redis
import random
from random import randint
from datetime import datetime, timedelta

### Users registration time

Here is a structure that groups users by the time they logged in. We have several users and for each of them we have the time of their registration as a timestamp.

In [2]:
redis_client = redis.Redis()

def random_timestamp(start_date, end_date):
    start_timestamp = start_date.timestamp()
    end_timestamp = end_date.timestamp()
    random_timestamp = random.uniform(start_timestamp, end_timestamp)
    return random_timestamp

start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 1, 1)

redis_client.zadd(
    "user-reg-time",
    {
        f"user {i}" : random_timestamp(start_date, end_date)
        for i in range(10)
    }
)

0

Now let's print out all the users created and their registration times.

In [3]:
user_reg_time = redis_client.zrange(
    "user-reg-time", 0, -1, withscores=True
)
for u, ts in user_reg_time:
    print(f"{u.decode('utf-8')} - {datetime.fromtimestamp(ts)}")

user 7 - 2023-01-10 09:49:36.410655
user 3 - 2023-03-27 18:26:17.201846
user 1 - 2023-04-22 17:04:14.173122
user 4 - 2023-06-15 00:17:52.191899
user 6 - 2023-07-16 07:48:01.785758
user 5 - 2023-07-24 17:36:54.901679
user 2 - 2023-08-20 00:39:49.093971
user 8 - 2023-09-21 04:09:57.631230
user 9 - 2023-10-17 12:02:05.911488
user 0 - 2023-11-24 16:31:20.298965


## Posts likes

Here are a bunch of posts created. Each post has a certain number of likes:

In [4]:
redis_client.zadd(
    "post-likes",
    {
        f"post {p}" : randint(50, 1000)
        for p in range(10)
    }
)

0

Now let's print out different posts and the number of likes they receive.

In [5]:
post_like = redis_client.zrange(
    "post-likes", 0, -1, withscores=True
)
for p, nl in post_like:
    print(f"{p.decode('utf-8')} - {nl}")

post 3 - 166.0
post 8 - 404.0
post 4 - 440.0
post 1 - 447.0
post 6 - 479.0
post 9 - 505.0
post 0 - 589.0
post 5 - 880.0
post 2 - 903.0
post 7 - 991.0


Don't forget to close `redis_client` when you are done with the Python code.

In [6]:
redis_client.close()

## zrange

Is a command that allows you to get the elements of the zset according to their ranks.

Let's say we need to get the 3 earliest registered users. To achieve it we can execute command from the following cell:

In [5]:
!redis-cli ZRANGE user-reg-time 0 2

1) "user 5"
2) "user 0"
3) "user 6"


By adding the `withscores` option to the `zrange` command, you can specify that you want to see the scores as well. The following example shows what you'll get:

In [6]:
!redis-cli ZRANGE user-reg-time 0 -1 withscores

 1) "user 5"
 2) "1677010879.0340476"
 3) "user 0"
 4) "1680636510.0739911"
 5) "user 6"
 6) "1680783743.4424393"
 7) "user 7"
 8) "1681288803.5887868"
 9) "user 9"
10) "1681815455.9353263"
11) "user 3"
12) "1686031151.2091238"
13) "user 4"
14) "1689847667.8113067"
15) "user 1"
16) "1689887979.0956852"
17) "user 2"
18) "1696444529.021455"
19) "user 8"
20) "1698984483.9752893"


## zrangebyscore

You can use the `zrangebyscore` command to get values that correspond to a range of scores.

So here is an option to get users who registered before 2023-06-01.

In [7]:
!redis-cli ZRANGEBYSCORE user-reg-time 0 $(date -d "2023-06-01" +%s)

1) "user 5"
2) "user 0"
3) "user 6"
4) "user 7"
5) "user 9"


## zscore

You can access socre by value using the `zscore` command.

In [11]:
!redis-cli zscore user-reg-time "user 9"

"1680923027.766844"


## zincrby

This command allows you to increase the score for a given value. The following example show how it works. 

So the next cell shows how it works. Here is the number of likes for a particular post, and then `zincrby` adds some likes to that post and prints it out again.

In [17]:
%%bash
redis-cli

ZSCORE post-likes "post 1"
ZINCRBY post-likes 20 "post 1"
ZSCORE post-likes "post 1"

447
467
467


The result is that the score has increased by the number of points specified.

## Delete examples

Don't forget to delete all crated entities to save globar redis clear.

In [13]:
!redis-cli DEL user-reg-time post-likes

(integer) 2
